## **Установка требуемых библиотек**

In [1]:
!pip install transformers datasets torch

### **Загрузка библиотек**

In [2]:
import torch

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### **Анализ датасета**

In [4]:
from datasets import load_dataset

In [5]:
dataset = load_dataset("MonoHime/ru_sentiment_dataset")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/276M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/32.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/189891 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21098 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'sentiment'],
        num_rows: 189891
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'sentiment'],
        num_rows: 21098
    })
})

In [7]:
dataset['train'][0]

{'Unnamed: 0': 21098,
 'text': '.с.,и спросил его:  о Посланник Аллаха!Ты порицаешь что-то из слушания?  Он ответил: я не порицаю ничего из него,но передай им,чтобы они открывали свои собрания Кораном и закрывали их Кораном ...........Это дошедшие до нас мнения и тот кто находится в поисках истины,по мере изучения этого вопроса будет сталкиваться с разногласиями и будет оставаться в растерянности или склонится к мнению одной из сторон по своему желанию.Но всего этого недостаточно,потому что он сам должен найти истину,подробно изучив вопросы запретного и разрешённого.|||||||||||||||||||||||||||||||||||||Обрати внимание:основатели всех четырёх мазхабов осуждали песни и поэззию.И этим часто грешат заблудшие суфии.Исключение делается для исламской тематики  Сахих БухариО проявлении радости во время праздника.498 (949). Сообщается, что ‘Аиша, да будет доволен ею Аллах, сказала: (Однажды) посланник Аллаха, да благословит его Аллах и приветствует, вошёл ко мне в то время, когда у меня находил

### **Разделение на тестовую и тренировочную подвыборки**

In [8]:
train_test = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test['train']
test_dataset = train_test['test']

### **Возьмем для начала датасет поменьше**

In [9]:
train_dataset = train_dataset.select(range(10000))
test_dataset = test_dataset.select(range(2000))

In [10]:
print(train_dataset.shape, test_dataset.shape)

(10000, 3) (2000, 3)


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [13]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
    num_labels=3)
model.to(device)
model.eval()

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [28]:
text = test_dataset["text"]
batch_size = 64
predictions = []

In [23]:
def tokenize(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [24]:
from torch.utils.data import DataLoader

test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [25]:
test_texts = test_dataset['text']
test_labels = test_dataset['sentiment']

In [29]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score, accuracy_score

for i in tqdm(range(0, len(test_texts), batch_size)):
    batch_texts = test_texts[i:i + batch_size]

    inputs = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        batch_predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
        predictions.extend(batch_predictions)

predictions = np.array(predictions)

100%|██████████| 32/32 [00:16<00:00,  1.99it/s]



📊 Вычисление метрик...


In [30]:
f1_macro = f1_score(test_labels, predictions, average='macro')
f1_weighted = f1_score(test_labels, predictions, average='weighted')

In [32]:
f1_macro, f1_weighted

(0.25890783768020686, 0.34205183479644785)

In [35]:
report = classification_report(test_labels, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.38      0.08      0.14       533
           1       0.48      0.94      0.64       957
           2       0.14      0.00      0.00       510

    accuracy                           0.47      2000
   macro avg       0.33      0.34      0.26      2000
weighted avg       0.37      0.47      0.34      2000

